<CENTER>
    <a href="http://opendata.atlas.cern/release/2020/documentation/notebooks/intro.html" class="icons"><img src="../../images/ATLASOD.gif" style="width:40%"></a>
</CENTER>

<CENTER><h1>Searching for the Higgs boson in the H&#8594;&gamma;&gamma; channel (using C++) </h1></CENTER>

## C++ notebook example


**Introduction**
Let's take a current ATLAS Open Data sample and create a histogram:

In order to activate the interactive visualisation of the histogram that is later created we can use the JSROOT magic:

In [ ]:
%jsroot on

We need to include some standard C++ and ROOT libraries

In [2]:
// Creates a TChain to be used by the Analysis.C class
#include <TChain.h>
#include <vector>
#include <TFile.h>
#include <iostream>
#include <string>
#include <stdio.h>
#include <chrono>

Because we would like to use more than one ROOT input file, the best option is to use a TChain object. This allows to "chain" several samples into a single structure that we can later loop over

In [3]:
//TString path = "https://opendata.cern.ch/eos/opendata/atlas/rucio/opendata/";
TString path = "root://eospublic.cern.ch:1094//eos/opendata/atlas/rucio/opendata/";

In [ ]:
TChain* fChain = new TChain("analysis");

TString prefix = "ODEO_FEB2025_v0";
TString skim = "GamGam";

vector<TString> samples = { "data15_periodD", "data15_periodE", "data15_periodF", "data15_periodG", 
                            "data15_periodH", "data15_periodJ", "data16_periodA", "data16_periodB", 
                            "data16_periodC", "data16_periodD", "data16_periodE", "data16_periodF", 
                            "data16_periodG", "data16_periodI", "data16_periodK", "data16_periodL"
                           };

cout << samples.size() << endl;

// Set the number of samples you want to include in the analysis (16 data samples in total)
int max_samples = 16;

for(int ii=0; ii<max_samples; ii++){
    TString name_sample = samples.at(ii);
    TString name_root_file = path+prefix+"_"+skim+"_"+name_sample+"."+skim+".root";
    cout << name_root_file << endl;
    fChain->Add(name_root_file);
}

int nentries = (Int_t)fChain->GetEntries();
cout << "Number of entries: " << nentries << endl;

Now we're going to extract the photons variables

In [5]:
Int_t  photon_n;  //number of preselected photons                                                                                                             

ROOT::VecOps::RVec<float> *photon_pt;  //transverse momentum of the photon                                                                                    
ROOT::VecOps::RVec<float> *photon_eta;  //pseudorapidity of the photon                                                                                        
ROOT::VecOps::RVec<float> *photon_phi;  //azimuthal angle of the photon                                                                                       
ROOT::VecOps::RVec<float> *photon_e;  //energy of the photon                                                                                                  
ROOT::VecOps::RVec<bool> *photon_isTightID;
ROOT::VecOps::RVec<bool> *photon_isTightIso;

Bool_t trigP;
ROOT::VecOps::RVec<float> *photon_ptcone20;

photon_pt = 0;
photon_eta = 0;
photon_phi = 0;
photon_e = 0;
photon_isTightID = 0;
photon_isTightIso = 0;
photon_ptcone20 = 0;

Here we're filling the variables defined above with the content of those inside the input ntuples

In [6]:
fChain->SetBranchAddress("photon_pt",        &photon_pt);
fChain->SetBranchAddress("photon_n",         &photon_n);

fChain->SetBranchAddress("photon_eta",       &photon_eta);
fChain->SetBranchAddress("photon_phi",       &photon_phi);
fChain->SetBranchAddress("photon_e",         &photon_e);
fChain->SetBranchAddress("photon_isTightID", &photon_isTightID);
fChain->SetBranchAddress("photon_isTightIso", &photon_isTightIso);

fChain->SetBranchAddress("trigP",            &trigP);
fChain->SetBranchAddress("photon_ptcone20",  &photon_ptcone20);

We're creating a histogram for this example. The plan in to fill them with events.

In [7]:
//Invariant mass histograms definition
TH1F *h_M_Hyy = new TH1F("h_M_Hyy","Diphoton invariant-mass ; M_{#gamma#gamma} [GeV] ; Events / bin", 60, 100, 160);

We are selecting below a simple look for them.

In [8]:
h_M_Hyy->SetMarkerSize(2.0);
h_M_Hyy->SetLineColor(kBlue);
h_M_Hyy->SetFillColor(kBlue-10);

The Higgs boson analysis implemented here considers Higgs boson decays into a photon pair. The event selection criteria are (this will take a few minutes):

In [9]:
auto start_time = chrono::steady_clock::now(); //Start the clock

int nbytes = 0;
int accepted_events = 0;
for(int i = 0; i < nentries; i++){
    nbytes = fChain->GetEntry(i);
    
    // printing the evolution in number of events
    if(i%10000000==0) cout << "Number of events analyzed: " << i << endl;
            
    if(photon_isTightID->at(0)==true && photon_isTightID->at(1)==true){
        // Cut: pT cut - photon 1 has pT > 50 GeV and photon 2 has pT > 30 GeV
        if(photon_pt->at(0) > 50. && photon_pt->at(1) > 30.){
            // Only the events where the calorimeter isolation is less than 5.5% are kept
            if(photon_ptcone20->at(0)/photon_pt->at(0) < 0.055 && photon_ptcone20->at(1)/photon_pt->at(1) < 0.055){
                // Cut on the pseudorapidity in barrel/end-cap transition region
                if((TMath::Abs(photon_eta->at(0)) < 2.37) && 
                   (TMath::Abs(photon_eta->at(0)) < 1.37 || TMath::Abs(photon_eta->at(0)) > 1.52)){
                    if((TMath::Abs(photon_eta->at(1)) < 2.37) && 
                       (TMath::Abs(photon_eta->at(1)) < 1.37 || TMath::Abs(photon_eta->at(1)) > 1.52)){
                        // TLorentzVector definitions
                        TLorentzVector Photon_1 = TLorentzVector();
                        TLorentzVector Photon_2 = TLorentzVector();

                        Photon_1.SetPtEtaPhiE(photon_pt->at(0), photon_eta->at(0), photon_phi->at(0), photon_e->at(0));
                        Photon_2.SetPtEtaPhiE(photon_pt->at(1), photon_eta->at(1), photon_phi->at(1), photon_e->at(1));
                        
                        // Calculation of the Invariant Mass using TLorentz vectors
                        TLorentzVector Photon_12 = Photon_1 + Photon_2;
                        float inv_mass_Hyy = Photon_12.M();

                        // Cut on null diphoton invariant mass
                        if(inv_mass_Hyy == 0) continue;
                        // Cut on diphoton invariant mass based isolation. Only the events where 
                        // the invididual photon invariant mass based isolation is larger than 35% are kept
                        if((photon_pt->at(0)/inv_mass_Hyy > 0.35) && (photon_pt->at(1)/inv_mass_Hyy > 0.35)){
                            // Filling with the mass of the gamma-gamma system
                            h_M_Hyy->Fill(inv_mass_Hyy);
                            accepted_events+=1;
                        }
                    }
                }
            }
        }
    }
}

// Final message after analysis
std::cout << "* Analysed a total of: " << nentries << " in this sample." << std::endl;
std::cout << "* Number of events passing the selection criteria: " << accepted_events << " in this sample." << std::endl;

Number of events analyzed: 0
Number of events analyzed: 10000000
Number of events analyzed: 20000000
Number of events analyzed: 30000000
* Analysed a total of: 36564144 in this sample.
* Number of events passing the selection criteria: 550057 in this sample.


#### Final plot

In [10]:
//*****************************************************************************************************************
// Fitting part
//*****************************************************************************************************************

using namespace RooFit;

// Convert histogram into RooDataHist
RooRealVar x("m_yy", "di-photon invariant mass m_{#gamma#gamma} [GeV]", 100, 160);
RooDataHist data_myy("data_myy", "data_myy", RooArgList(x), h_M_Hyy);

// --- Define signal model (Gaussian) ---
RooRealVar mean("mean", "mean of gauss", 125, 120, 130);
RooRealVar sigma("sigma", "width of gauss", 2.0, 0.5, 5.0);
RooGaussian gauss("gauss", "gaussian PDF", x, mean, sigma);

// --- Define background model (4th order polynomial) ---
RooRealVar a0("a0", "a0", 0.0, -1, 1);
RooRealVar a1("a1", "a1", 0.0, -1, 1);
RooRealVar a2("a2", "a2", 0.0, -1, 1);
RooRealVar a3("a3", "a3", 0.0, -1, 1);
RooRealVar a4("a4", "a4", 0.0, -1, 1);
RooChebychev bkg("bkg", "Chebychev background", x, RooArgList(a0,a1,a2,a3,a4));

// --- Define coefficients (yields) ---
RooRealVar nsig("nsig", "signal yield", h_M_Hyy->Integral()/20., 0., h_M_Hyy->Integral());
RooRealVar nbkg("nbkg", "background yield", h_M_Hyy->Integral(), 0., h_M_Hyy->Integral());

// --- Total model ---
RooAddPdf model("model", "sig + bkg", RooArgList(gauss, bkg), RooArgList(nsig, nbkg));

// --- Perform the fit ---
model.fitTo(data_myy, RooFit::Extended(true), RooFit::Save(), RooFit::PrintLevel(-1));

std::cout << "mean: " << mean.getVal() << endl;
std::cout << "std deviation: " << sigma.getVal() << endl;
std::cout << "polynomic function (using the Chebyshev polynomials basis):" << endl;
std::cout << a0.getVal() << "+" << a1.getVal() << "*T_1(x)+" << a2.getVal() << "*T_2(x)+" << a3.getVal() << "*T_3(x)+" << a4.getVal() << "*T_4(x)" << endl; 

[#1] INFO:Fitting -- RooAbsPdf::fitTo(model) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using CPU computation library compiled with -mavx512
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_data_myy) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: deactivating const optimization
mean: 124.728
std deviation: 1.3945
polynomic function (using the Chebyshev polynomials basis):
-0.73128+0.0992972*T_1(x)+0.0180945*T_2(x)+-0.0119551*T_3(x)+0.00462275*T_4(x)


In [11]:
// =======================
// Main Pad (Data + Fit)
// =======================

TCanvas* canvas = new TCanvas("canvas", "canvas", 800, 600);

TPad* mainPad = new TPad("mainPad", "mainPad", 0, 0.3, 1, 1.0);
mainPad->SetBottomMargin(0);
mainPad->Draw();
mainPad->cd();

// Get histogram representation of data and background
// Ensure both histograms use the same binning

int nbins = h_M_Hyy->GetNbinsX();
double xmin = h_M_Hyy->GetXaxis()->GetXmin();
double xmax = h_M_Hyy->GetXaxis()->GetXmax();

TH1* h_data = data_myy.createHistogram("h_data", x, RooFit::Binning(nbins, xmin, xmax));
TH1* h_bkg  = bkg.createHistogram("h_bkg", x, RooFit::Binning(nbins, xmin, xmax));

std::cout << "nbins h_data: " << h_data->GetNbinsX() << endl;
std::cout << "nbins h_bkg: " << h_bkg->GetNbinsX() << endl;

std::cout << "===================================================================" << endl;
std::cout << "integral h_bkg: " << h_bkg->Integral() << endl;
std::cout << "nbkg expected: " << nbkg << endl;

// Normalize background histogram to expected number of events
h_bkg->Scale(nbkg.getVal() / h_bkg->Integral());

// Create residual histogram (data - bkg)
TH1F* resid = new TH1F("residual","Diphoton invariant-mass ; Invariant Mass m_{yy} [GeV] ; events",60,100,160);

for(int i = 1; i <= h_data->GetNbinsX(); i++) {
    float data_value = h_data->GetBinContent(i);
    float bkg_value = h_bkg->GetBinContent(i);

    // residual value
    float res_value  = data_value - bkg_value;
    resid->SetBinContent(i, res_value);
}

// --- Plot results ---
RooPlot* frame = x.frame();
data_myy.plotOn(frame, RooFit::MarkerStyle(20), RooFit::MarkerSize(0.5));
model.plotOn(frame, RooFit::Name("sig_bkg_curve"), RooFit::LineWidth(2), 
             RooFit::LineColor(kBlue));
model.plotOn(frame, RooFit::Components("bkg"), RooFit::LineWidth(2), RooFit::LineStyle(kDashed), 
             RooFit::LineColor(kRed), RooFit::Name("bkg_curve"));
model.plotOn(frame, RooFit::Components("gauss"), RooFit::LineStyle(kDotted), 
             RooFit::LineColor(kBlue), RooFit::Name("gaussian_curve"));

frame->SetTitle("Di-photon invariant mass m_{#gamma#gamma}");
frame->GetYaxis()->SetTitle("Events / bin");
frame->GetXaxis()->SetTitleSize(0.05);   // size of x-axis title
frame->GetYaxis()->SetTitleSize(0.05);   // size of y-axis title
frame->GetXaxis()->SetLabelSize(0.03);   // size of x-axis labels
frame->GetYaxis()->SetLabelSize(0.03);   // size of y-axis labels
frame->GetXaxis()->SetTitleFont(42);     // Helvetica (ROOT default)
frame->GetYaxis()->SetTitleFont(42);

frame->Draw();

canvas->cd();

// Legend
RooCurve* sig_bkg_curve = (RooCurve*) frame->findObject("sig_bkg_curve");
RooCurve* bkg_curve = (RooCurve*) frame->findObject("bkg_curve");

TLegend* legend = new TLegend(0.55, 0.75, 0.85, 0.90);
legend->AddEntry(frame, "Data", "lep");
legend->AddEntry(sig_bkg_curve, "Sig+Bkg Fit (m_{H}=125 GeV)", "l");
legend->AddEntry(bkg_curve, "Bkg (4th order polynomial)", "l");
legend->Draw();

// Labels
TLatex* latex = new TLatex();
latex->SetTextSize(0.03);
latex->DrawLatexNDC(0.60, 0.70, "ATLAS Open Data");
latex->DrawLatexNDC(0.60, 0.65, "#sqrt{s}=13 TeV, #int L dt = 36 fb^{-1}");
latex->DrawLatexNDC(0.60, 0.60, "H #rightarrow #gamma#gamma");

// =======================
// Residual Pad (Data-Bkg vs Fit-Bkg)
// =======================
TPad* residPad = new TPad("residPad", "residPad", 0, 0.05, 1, 0.3);
residPad->SetTopMargin(0);
residPad->SetBottomMargin(0.3);
residPad->Draw();
residPad->cd();

// Style
resid->SetMarkerStyle(20);
resid->SetMarkerSize(0.5);
resid->SetLineColor(kBlack);
resid->GetYaxis()->SetTitle("Events - Bkg");
resid->GetYaxis()->SetTitleSize(0.10);
resid->GetYaxis()->SetTitleOffset(0.5);
resid->GetYaxis()->SetLabelSize(0.10);
resid->GetXaxis()->SetTitle("di-photon invariant mass m_{#gamma#gamma} [GeV]");
resid->GetXaxis()->SetTitleSize(0.10);
resid->GetXaxis()->SetLabelSize(0.10);
resid->SetStats(0);

// Draw residuals (data - bkg)
resid->Draw("E1");

// Zero line
TLine* zeroLine = new TLine(100, 0, 160, 0);
zeroLine->SetLineColor(kRed);
zeroLine->SetLineStyle(2);
zeroLine->Draw("same");

RooCurve* gaussian_curve = (RooCurve*) frame->findObject("gaussian_curve");
gaussian_curve->Draw("same");

canvas->Draw();

nbins h_data: 60
nbins h_bkg: 60
integral h_bkg: 0.99999
nbkg expected: 249108
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (bkg)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: ()
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (gauss)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: ()


In [12]:
auto end_time = chrono::steady_clock::now();
auto elapsed = chrono::duration_cast<chrono::seconds>(end_time - start_time).count();
cout << "Time taken: " << elapsed/60. << " min." << endl;

Time taken: 9.13333 min.


**Done!**